In [201]:
import tensorflow as tf
from keras import layers as klayer
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [202]:
tf.random.set_seed(1234)

print("--Get data--")
data = pd.read_csv('heart.csv')
data.set_index(['row.names'],inplace=True)
data.convert_dtypes()

--Get data--


,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
row.names,,,,,,,,,,
1,160,12.0,5.73,23.11,Present,49,25.3,97.2,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.5,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.6,3.5,27.78,Present,60,25.99,57.34,49,1
...,...,...,...,...,...,...,...,...,...,...
459,214,0.4,5.98,31.72,Absent,64,28.45,0.0,58,0
460,182,4.2,4.41,32.1,Absent,52,28.61,18.72,52,1
461,108,3.0,1.59,15.23,Absent,40,20.09,26.64,55,0


In [203]:
#Change text data to numerical data
def convert_famhist (text):
	if text == 'Present':
		return 1 
	else:
		return 0


In [204]:
# apply convert function to dataframe
data['famhist_flaged'] = data['famhist'].apply(convert_famhist)
print(data.dtypes)
data.head(10)


sbp                 int64
tobacco           float64
ldl               float64
adiposity         float64
famhist            object
typea               int64
obesity           float64
alcohol           float64
age                 int64
chd                 int64
famhist_flaged      int64
dtype: object


,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd,famhist_flaged
row.names,,,,,,,,,,,
1,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1,0
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0,1
4,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1,1
5,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1,1
6,132,6.20,6.47,36.21,Present,62,30.77,14.14,45,0,1
7,142,4.05,3.38,16.20,Absent,59,20.81,2.62,38,0,0
8,114,4.08,4.59,14.60,Present,62,23.11,6.72,58,1,1
9,114,0.00,3.83,19.40,Present,49,24.86,2.49,29,0,1


In [205]:
# Split the dataset to training set and test set
train_set = data.sample(frac=0.8,random_state=0,axis=0,replace=0)
train_set.to_csv('heart_train.csv')
test_set = data[~data.index.isin(train_set.index)]
test_set.to_csv('heart_test.csv')

In [206]:
x_train = train_set.loc[:,['sbp',	'tobacco','ldl','adiposity','typea', 'obesity',	'alcohol','age','famhist_flaged']]
x_test = test_set.loc[:,['sbp',	'tobacco','ldl','adiposity','typea', 'obesity',	'alcohol','age','famhist_flaged']]
y_train = train_set.loc[:,['chd']]
y_test = test_set.loc[:,['chd']]


In [219]:
print("--Make model--")
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation = "relu", use_bias= True, kernel_regularizer=tf.keras.regularizers.l2(0.0001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(512, activation = "relu", use_bias= True))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print("--Fit model--")
model.fit(x_train, y_train, epochs=30, verbose=1)
print("--Evaluate model--")
model_loss1, model_acc1 = model.evaluate(x_train,  y_train, verbose=2)
model_loss2, model_acc2 = model.evaluate(x_test,  y_test, verbose=2)
print(f"Train / Test Accuracy: {model_acc1*100:.1f}% / {model_acc2*100:.1f}%")

--Make model--
--Fit model--
Epoch 1/30
12/12 [==============================] - 1s 3ms/step - loss: 20.8852 - accuracy: 0.4703
Epoch 2/30
12/12 [==============================] - 0s 5ms/step - loss: 2.6040 - accuracy: 0.5432
Epoch 3/30
12/12 [==============================] - 0s 4ms/step - loss: 0.8351 - accuracy: 0.5919
Epoch 4/30
12/12 [==============================] - 0s 4ms/step - loss: 0.6246 - accuracy: 0.6514
Epoch 5/30
12/12 [==============================] - 0s 4ms/step - loss: 0.6085 - accuracy: 0.6270
Epoch 6/30
12/12 [==============================] - 0s 5ms/step - loss: 0.6124 - accuracy: 0.6378
Epoch 7/30
12/12 [==============================] - 0s 5ms/step - loss: 0.6234 - accuracy: 0.6432
Epoch 8/30
12/12 [==============================] - 0s 4ms/step - loss: 0.6076 - accuracy: 0.6568
Epoch 9/30
12/12 [==============================] - 0s 4ms/step - loss: 0.5909 - accuracy: 0.6649
Epoch 10/30
12/12 [==============================] - 0s 5ms/step - loss: 0.5896 - accura